In [2]:
import requests

from collections import Counter
import operator
import nltk
from bs4 import BeautifulSoup

import pickle
import xmltodict, json
import time

import numpy as np
import pandas as pd

import re

In [3]:
grkey = pickle.load( open( "grkey.p", "rb" ) )

In [4]:
# Main page thread: 18661546
main_page = requests.request('GET', 'https://hacker-news.firebaseio.com/v0/item/18661546.json').json()

In [5]:
text = []
i  = 0
kids = main_page["kids"]
kidspast = main_page["kids"]
while (kids != []) and (len(kidspast)<main_page['descendants']-1):
    kidsnew = []
    for kid in kids:
        i += 1
        item = requests.request('GET', 'https://hacker-news.firebaseio.com/v0/item/{kid}.json'.format(kid=kid)).json()
        try:
            soup = BeautifulSoup(item['text'])
            text.append(soup.findAll(text=True))
        except:
            pass
        try:
            kidsnew += item['kids']
        except:
            pass
    kidspast += list(set(kidsnew))
    kidspast = list(set(kidspast))
    kids = [k for k in list(set(kidsnew)) if k not in kidspast]
    
flat_list = [item for sublist in text for item in sublist]

/Users/am8e13/anaconda/lib/python2.7/site-packages/bs4/__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


In [6]:
pickle.dump(text, open( "text.p", "wb" ))

In [7]:
text = pickle.load( open( "text.p", "rb" ) )

In [8]:
text_clean = [re.sub(r"[^a-zA-Z0-9]+", ' ', k)  for t in flat_list for k in t.split("\n")]

In [10]:
countsb = Counter()
countst = Counter()
countsq = Counter()
countsc = Counter()

words = re.compile(r'\w+')
for t in text_clean:
    w = words.findall(t.lower())
    countsb.update(zip(w,w[1:]))
    countst.update(zip(w,w[1:],w[2:]))
    countsq.update(zip(w,w[1:],w[2:],w[3:]))
    countsc.update(zip(w,w[1:],w[2:],w[3:],w[4:]))

In [11]:
results = {}

# save the results
results[1] = sorted(
    countsb.items(),
    key=operator.itemgetter(1),
    reverse=True
)

results[2] = sorted(
    countst.items(),
    key=operator.itemgetter(1),
    reverse=True
)

results[3] = sorted(
    countsq.items(),
    key=operator.itemgetter(1),
    reverse=True
)

results[4] = sorted(
    countsc.items(),
    key=operator.itemgetter(1),
    reverse=True
)

In [13]:
stops = ["i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", "yours", "yourself", 
         "yourselves", "he", "him", "his", "himself", "she", "her", "hers", "herself", "it", "its", 
         "itself", "they", "them", "their", "theirs", "themselves", "what", "which", "who", "whom", 
         "this", "that", "these", "those", "am", "is", "are", "was", "were", "be", "been", "being", 
         "have", "has", "had", "having", "do", "does", "did", "doing", "and", "an", "a", "the",  "but", 
         "if", "or", "because", "as", "until", "while", "of", "at", "by", "for", "with", "about", "against", 
         "between", "into", "through", "during", "before", "after", "above", "below", "to", "from", "up", 
         "down", "in", "out", "on", "off", "over", "under", "again", "further", "then", "once", "here", 
         "there", "when", "where", "why", "how", "all", "any", "both", "each", "few", "more", "most", 
         "other", "some", "such", "no", "nor", "not", "only", "own", "same", "so", "than", "too", "very", 
         "s", "t", "can", "will", "just", "don", "should", "now", 'https', "www", "goodreads", "didn", "isn", "doesn"]

In [14]:
df = {}
for k in [1,2,3,4]:
    names = []
    counts = []
    for c in results[k]:
        lst = list(c[0])
        lst = [w for w in lst if ((w not in stops) and w.isalpha())]
        if c[1]>2 & len(lst)>0:
            names.append(' '.join(c[0]))
            counts.append(c[1])
    df[k] = pd.DataFrame(data={'names':names, 'counts':counts})
    print(len(df[k]))

146
117
56
20


In [ ]:
for jj in [1,2,3,4]:
    df[jj]['gr_title'] = None
    df[jj]['gr_author'] = None
    
    for it in df[jj].iterrows():
        name = it[1]['names']
        res = requests.get("https://www.goodreads.com/search/index.xml" , params={"key": grkey, "q":name})
        xpars = xmltodict.parse(res.text)
        json1 = json.dumps(xpars)
        d = json.loads(json1) 

        if d['GoodreadsResponse']['search']['results'] is None:
            title = None
            author = None
        elif type(d['GoodreadsResponse']['search']['results']['work']) == dict:
            title = d['GoodreadsResponse']['search']['results']['work']['best_book']['title']
            author = d['GoodreadsResponse']['search']['results']['work']['best_book']['author']['name']
        else:
            lst = d['GoodreadsResponse']['search']['results']['work']
            ys = []
            for j in range(len(lst)): 
                if '#text' in lst[j]['original_publication_year'].keys():
                    #avg_rat.append(lst[j]['average_rating'])
                    ys.append(int(lst[j]['original_publication_year']['#text']))
                    #rat_count.append(lst[j]['ratings_count'])
                else:
                    ys.append(1)
            j = np.argmax(ys)    
            title = lst[j]['best_book']['title']
            author = lst[j]['best_book']['author']['name']

        df[jj]['gr_title'].loc[it[0]] = title
        df[jj]['gr_author'].loc[it[0]] = author
        time.sleep(1)

In [530]:
lst = []
for j in [1,2,3,4]:
    lst += list(set(df[j]['gr_title']))
lst = list(set(lst))

In [531]:
books = []
for j in lst:
    if j != None:
        b = j.split(':')[0].split('(')[0]
        books.append(b)

In [532]:
conts = []
for b in books:
    count = 0
    for f in flat_list:
        if f.lower().find(b.lower()) >= 0:
            count += 1
    conts.append(count)
print(sum(conts))

117


In [533]:
dfp = pd.DataFrame(data={'books':books, 'counts':conts})

In [534]:
dfp[dfp['counts']>1].sort_values('counts', ascending=False)

,books,counts
104,Bad Blood,12
19,Magi,7
53,Why We Sleep,7
165,How to Change Your Mind,6
153,Shoe Dog,6
89,Man's Search for Meaning,5
157,Factfulness,5
139,Deep Work,5
24,Thinking in Systems,4
156,21 Lessons for the 21st Century,4
